In [1]:
import time

import adaptive
import numpy as np
from qcodes import Instrument, ManualParameter
from scipy import optimize

import quantify_core.analysis.optimization_analysis as oa
import quantify_core.visualization.pyqt_plotmon as pqm
from quantify_core.analysis.interpolation_analysis import InterpolationAnalysis2D
from quantify_core.data.handling import set_datadir, default_datadir
from quantify_core.measurement.control import MeasurementControl
from quantify_core.visualization.instrument_monitor import InstrumentMonitor

In [2]:
set_datadir(default_datadir())

Data will be saved in:
/home/pschaefers/quantify-data


In [3]:
meas_ctrl = MeasurementControl("meas_ctrl")

In [4]:
para = Instrument("parabola")

para.add_parameter("freq", unit="Hz", label="Frequency", parameter_class=ManualParameter)
para.add_parameter("ampl", unit="V", label="Amplitude", parameter_class=ManualParameter)
para.add_parameter("dur", unit="s", label="Duration", parameter_class=ManualParameter)

# para.add_parameter(
#     "noise", unit="V", label="white noise amplitude", parameter_class=ManualParameter
# )
# para.add_parameter(
#     "acq_delay", initial_value=0.1, unit="s", parameter_class=ManualParameter
# )


def _amp_model():
    time.sleep(
        para.acq_delay()
    )  # for display purposes, just so we can watch the live plot update
    return -para.x() ** 2 - para.y() ** 2 - para.z**2


para.add_parameter("F", unit="[-]", label="Fidelity", get_cmd=_amp_model)

<qcodes.parameters.parameter.Parameter: F at 140243144577888>

In [ ]:
def CMAES_adaptive(func, Ngen, popsize, optimizer, deltaFreq, minFreq, deltaAmpl, minAmpl, deltaDur, minDur):
    for generation in range(Ngen):  
        normalized_mean = optimizer.mean
        mean = np.copy(normalized_mean)
        
        mean[0] = mean[0] * deltaFreq + minFreq
        mean[1] = mean[1] * deltaAmpl + minAmpl
        mean[2] = mean[2] * deltaDur + minDur
    
        normalized_freq = np.zeros(popsize)
        normalized_ampl = np.zeros(popsize)
        normalized_dur = np.zeros(popsize)
        
        frequencies = np.zeros(popsize)
        amplitudes = np.zeros(popsize)
        durations = np.zeros(popsize)
        
        for i in range(popsize):
            x = optimizer.ask()
            
            normalized_freq[i] = x[0]
            normalized_ampl[i] = x[1]
            normalized_dur[i] = x[2]

            frequencies[i] = x[0]*deltaFreq + minFreq
            amplitudes[i] = x[1]*deltaAmpl + minAmpl
            durations[i] = x[2]*deltaDur + minDur
                    
        normalized_freq_ampl_and_dur = [(float(xi), float(yi), float(zi)) for xi, yi, zi in zip(normalized_freq, normalized_ampl, normalized_dur)]
        freq_ampl_and_dur = np.array([np.array([xi, yi, zi]) for xi, yi, zi in zip(frequencies, amplitudes, durations)])
        
        # objective_value = objective(freq_ampl_and_dur, meas_freq, meas_ampl, meas_dur, meas_fid) #This needs to be from fun 
        #I think it shoulde be:
        objective_value = func()
        
        solutions = [(wi, float(vi)) for wi, vi in zip(normalized_freq_ampl_and_dur, objective_value)]
        optimizer.tell(solutions)

In [ ]:
meas_ctrl.settables([para.x, para.y])
af_pars = {
    "adaptive_function": optimize.minimize,  # used by meas_ctrl
    "x0": [0.5,0.5],  # used by `optimize.minimize` (in this case)
    "method": "Nelder-Mead",  # used by `optimize.minimize` (in this case)
    "options": {"maxfev": 100},  # limit the maximum evaluations of the gettable(s)
}
para.noise(0.5)
meas_ctrl.gettables(para.amp)
dset = meas_ctrl.run_adaptive("nelder_mead_optimization", af_pars)

In [ ]:
def CMAES_adaptive_Old(fun, popsize, optimizer, deltaFreq, minFreq, deltaAmpl, minAmpl, deltaDur, minDur): #optimizer contains the bounds and fun is the gettable (should be the objective function)
    normalized_mean = optimizer.mean
    mean = np.copy(normalized_mean)
    
    mean[0] = mean[0] * deltaFreq + minFreq
    mean[1] = mean[1] * deltaAmpl + minAmpl
    mean[2] = mean[2] * deltaDur + minDur

    normalized_freq = np.zeros(popsize)
    normalized_ampl = np.zeros(popsize)
    normalized_dur = np.zeros(popsize)
    
    frequencies = np.zeros(popsize)
    amplitudes = np.zeros(popsize)
    durations = np.zeros(popsize)

     for i in range(popsize):
        x = optimizer.ask()
        
        normalized_freq[i] = x[0]
        normalized_ampl[i] = x[1]
        normalized_dur[i] = x[2]

        frequencies[i] = x[0]*deltaFreq + minFreq
        amplitudes[i] = x[1]*deltaAmpl + minAmpl
        durations[i] = x[2]*deltaDur + minDur
                
    normalized_freq_ampl_and_dur = [(float(xi), float(yi), float(zi)) for xi, yi, zi in zip(normalized_freq, normalized_ampl, normalized_dur)]
    freq_ampl_and_dur = np.array([np.array([xi, yi, zi]) for xi, yi, zi in zip(frequencies, amplitudes, durations)])
    objective_value = objective(freq_ampl_and_dur, meas_freq, meas_ampl, meas_dur, meas_fid)

    solutions = [(wi, float(vi)) for wi, vi in zip(normalized_freq_ampl_and_dur, objective_value)]
    optimizer.tell(solutions)

    return freq_ampl_dur

    
        